In [1]:
from scipy.io import wavfile
import cPickle as pickle
import sys
import os
import numpy as np

In [24]:
(0.5 * 16000 * 20) / 16000

10.0

In [26]:
test_start = 0.9
valid_start = 0.8

data_folder = os.environ["DATA_DIR"]

fs, data = wavfile.read(data_folder + os.path.sep + "data.wav")
data = np.asarray(data, dtype="float32")

# if x_length == 0.5, then
# x_length*fs = 16000*0.5 = 8000
# so an x^i represents 0.5 of a second
x_length = 0.5
print "each x will represent %f samples (%f seconds)" % (x_length*fs, x_length)
# a sequence is x_length*fs*seq_length long
seq_length = 20
print "each sequence will represent %f seconds" % ((0.5 * fs * 20) / fs)

how_many_seconds = 60*20 # 20 minutes
print "truncating input data to %i seconds" % (how_many_seconds)

if how_many_seconds > 0:
    sys.stderr.write("truncating data to %i seconds\n" % how_many_seconds)
    data = data[0: how_many_seconds*fs]

each x will represent 8000.000000 samples (0.500000 seconds)
each sequence will represent 10.000000 seconds
truncating input data to 1200 seconds


truncating data to 1200 seconds


In [27]:
# e.g. 0 -> 0.8
train_data = data[ 0 : len(data)*valid_start ]
# e.g. 0.8 -> 0.9 
valid_data = data[ len(data)*valid_start : len(data)*test_start ]
# e.g. 0.9 ::
test_data = data[ len(data)*test_start :: ]

min_ = np.min(data)
max_ = np.max(data)
train_data = (train_data - min_) / (max_ - min_)
valid_data = (valid_data - min_) / (max_ - min_)
test_data = (test_data - min_) / (max_ - min_)

dd = [train_data, valid_data, test_data]

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [39]:
for i in range(0, len(dd)):
    b = 0
    x_size = int(x_length*fs)
    batches = []
    seq = []
    while True:
        if b*x_size >= dd[i].shape[0]:
            break
        this_x = dd[i][b*x_size : (b+1)*x_size]
        seq.append(this_x)
        if len(seq) == seq_length:
            batches.append(seq)
            seq = []
        b += 1
    dd[i] = np.asarray(batches, dtype="float32")

In [41]:
for elem in dd:
    print elem.shape

(96, 20, 8000)
(12, 20, 8000)
(12, 20, 8000)
